In [2]:
import os
import pandas as pd
import numpy as np
#import xgboost
import utils
import scoring
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.regularizers import l2
from keras.optimizers import Adam
import tensorflow as tf
from keras import layers
import keras


In [3]:
columns =  ["id", "label", "weight", "sWeight", "kinWeight"]
FOI_Names = ['closest_x_0','closest_x_1','closest_x_2','closest_x_3','closest_y_0','closest_y_1','closest_y_2','closest_y_3',
            'closest_T_0','closest_T_1','closest_T_2','closest_T_3','closest_z_0','closest_z_1','closest_z_2','closest_z_3',
            'closest_dx_0','closest_dx_1','closest_dx_2','closest_dx_3','closest_dy_0','closest_dy_1','closest_dy_2','closest_dy_3']
# DATA_PATH = "."
DATA_PATH = "/Users/mykola/MLHEP/MLHEP-2020-muon-id"
train = pd.read_csv(os.path.join(DATA_PATH, "train.csv.gz"),index_col="id", usecols=columns+utils.SIMPLE_FEATURE_COLUMNS)
foi_train = pd.read_csv(os.path.join(DATA_PATH, "transformed_foi.csv"))

/usr/local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
col =  [ "label", "weight", "sWeight", "kinWeight"]

print(foi_train)

         Unnamed: 0  closest_x_0   closest_x_1   closest_x_2   closest_x_3  \
0                 0  8110.418500  12362.430000  17592.045000  33490.310000   
1                 1  2565.165300   4954.634000  17085.285000  11960.113000   
2                 2    27.115717     41.721634   2870.406200    408.955800   
3                 3    67.807945      7.291055    675.428830    790.541930   
4                 4     9.596987     19.489336  13997.559000  10585.928000   
...             ...          ...           ...           ...           ...   
1263649     1263649    12.909776    143.347430      8.835234    872.462460   
1263650     1263650    26.929157     60.519360    123.140366    495.869500   
1263651     1263651    39.359080    115.675385   1000.000000  36999.920000   
1263652     1263652  1445.503300   4386.151400   3480.251000     40.008095   
1263653     1263653   132.106860      2.306750    319.738220     24.365648   

          closest_y_0  closest_y_1   closest_y_2   closest_y_3 

In [5]:
nn_input = pd.concat([train, foi_train], axis=1, sort=False)

In [6]:
sc = StandardScaler()
nn_input.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names] = sc.fit_transform(nn_input.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names])
print (nn_input.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names])


         ncl[0]  ncl[1]  ncl[2]  ncl[3]  avg_cs[0]  avg_cs[1]  avg_cs[2]  \
0            79      18      22      10   2.392405   2.055556   1.590909   
1            33      14       6      15   1.575758   2.285714   1.333333   
2           115      53      23      25   2.843478   2.433962   1.826087   
3            28       8       5      15   2.964286   1.500000   1.800000   
4            50       8       4      13   2.400000   1.250000   1.500000   
...         ...     ...     ...     ...        ...        ...        ...   
1263649      40      23      12      12   2.325000   1.391304   1.166667   
1263650      34       8       8       9   3.323529   1.500000   1.625000   
1263651      25       2       9      10   1.480000   1.000000   1.222222   
1263652      56      11       5       6   4.607143   1.727273   2.800000   
1263653     141      42      23      33   2.411348   2.119048   1.478261   

         avg_cs[3]  ndof  MatchedHit_TYPE[0]  ...  closest_z_2  closest_z_3  \
0       

In [7]:
from keras import backend as K
def constrained_MSE(y_true,y_pred):
    return K.mean(K.square(tf.sigmoid(y_pred) - y_true))

In [16]:
import tensorlayer as tl
model = Sequential()
model.add(Dense(1024, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal', input_dim=89))
model.add(layers.Dropout(rate=0.45))
model.add(Dense(512, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal'))
model.add(layers.Dropout(rate=0.35))
#model.add(layers.BatchNormalization())
model.add(Dense(256, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal'))
model.add(layers.Dropout(rate=0.25))
#model.add(layers.BatchNormalization())
model.add(Dense(256, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal'))
model.add(layers.Dropout(rate=0.2))
#model.add(layers.BatchNormalization())
model.add(Dense(128, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal'))
model.add(layers.Dropout(rate=0.2))
#model.add(layers.BatchNormalization())
model.add(Dense(128, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal'))
#model.add(layers.Dropout(rate=0.2))
model.add(layers.BatchNormalization())
model.add(Dense(64, activation= lambda x : tl.act.lrelu(x, 0.1), kernel_initializer='random_normal'))
model.add(layers.BatchNormalization())
#model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#model.compile(optimizer =Adam(lr=0.001),loss='binary_crossentropy', metrics =['accuracy','binary_crossentropy'],weighted_metrics=['accuracy'])
model.compile(optimizer =Adam(lr=0.001),loss='binary_crossentropy', metrics =['accuracy'])

In [8]:
train_part, validation = train_test_split(nn_input, test_size=0.05, shuffle=True, random_state=2342234)
print(train_part.shape)
print(validation.shape)


(1200471, 94)
(63183, 94)


In [8]:
print(validation.label)


1130311    0
819219     0
155811     1
407242     1
164689     0
          ..
797663     0
302199     1
496404     1
1028939    1
397432     1
Name: label, Length: 63183, dtype: int64


In [114]:
save_all_xy = keras.callbacks.ModelCheckpoint("NNmodels/"+'weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#history = model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values, train_part.label, verbose=1, callbacks=[save_all_xy], validation_data=(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values, validation.label),  shuffle=True, batch_size=700, epochs=160)
history = model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values, train_part.sWeight, verbose=1, callbacks=[save_all_xy], validation_data=(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values, validation.sWeight),  shuffle=True, batch_size=500, epochs=200)
#history = model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train_part.label, verbose=1, callbacks=[save_all_xy],  sample_weight=train_part.weight.values, validation_data=(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, validation.label),  shuffle=True, batch_size=700, epochs=500)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Training loss', 'Validation loss'], loc='upper right')
plt.savefig("loss.png")
plt.show()

Epoch 1/200
2387/2401 [============================>.] - ETA: 0s - loss: 0.3867 - accuracy: 0.0000e+00 - binary_crossentropy: 10.3816
Epoch 00001: saving model to NNmodels/weights.01-0.38.h5
2401/2401 [==============================] - 6s 2ms/step - loss: 0.3867 - accuracy: 0.0000e+00 - binary_crossentropy: 10.4018 - val_loss: 0.3850 - val_accuracy: 0.0000e+00 - val_binary_crossentropy: 13.3058
Epoch 2/200
2389/2401 [============================>.] - ETA: 0s - loss: 0.3866 - accuracy: 0.0000e+00 - binary_crossentropy: 15.0672
Epoch 00002: saving model to NNmodels/weights.02-0.38.h5
2401/2401 [==============================] - 5s 2ms/step - loss: 0.3867 - accuracy: 0.0000e+00 - binary_crossentropy: 15.0488 - val_loss: 0.3850 - val_accuracy: 0.0000e+00 - val_binary_crossentropy: 12.6085
Epoch 3/200
2387/2401 [============================>.] - ETA: 0s - loss: 0.3865 - accuracy: 0.0000e+00 - binary_crossentropy: 16.3084
Epoch 00003: saving model to NNmodels/weights.03-0.38.h5
2401/2401 [==

KeyboardInterrupt: 

In [27]:
eval_model=model.evaluate(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values, validation.label)
print(eval_model)

1975/1975 [==============================] - 3s 2ms/step - loss: 0.2281 - accuracy: 0.6676 - binary_crossentropy: 3.7246 - weighted_accuracy: 0.6676
[0.228145033121109, 0.6676163077354431, 3.7246248722076416, 0.6676163077354431]


In [11]:
dirname = "/Users/mykola/Google Drive/NNmodels/1000_epochs/"
#dirname = "/Users/mykola/MLHEP/MLHEP-2020-muon-id/NNmodels"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(dirname) if isfile(join(dirname, f))]
print (sorted(onlyfiles))

['weights.01-0.42.h5', 'weights.02-0.40.h5', 'weights.03-0.41.h5', 'weights.04-0.40.h5', 'weights.05-0.40.h5', 'weights.06-0.40.h5', 'weights.07-0.39.h5', 'weights.08-0.40.h5', 'weights.09-0.39.h5', 'weights.10-0.39.h5', 'weights.100-0.38.h5', 'weights.1000-0.37.h5', 'weights.101-0.38.h5', 'weights.102-0.38.h5', 'weights.103-0.38.h5', 'weights.104-0.38.h5', 'weights.105-0.38.h5', 'weights.106-0.38.h5', 'weights.107-0.38.h5', 'weights.108-0.38.h5', 'weights.109-0.38.h5', 'weights.11-0.39.h5', 'weights.110-0.38.h5', 'weights.111-0.38.h5', 'weights.112-0.38.h5', 'weights.113-0.38.h5', 'weights.114-0.38.h5', 'weights.115-0.38.h5', 'weights.116-0.38.h5', 'weights.117-0.38.h5', 'weights.118-0.38.h5', 'weights.119-0.38.h5', 'weights.12-0.39.h5', 'weights.120-0.38.h5', 'weights.121-0.38.h5', 'weights.122-0.38.h5', 'weights.123-0.38.h5', 'weights.124-0.38.h5', 'weights.125-0.38.h5', 'weights.126-0.38.h5', 'weights.127-0.38.h5', 'weights.128-0.38.h5', 'weights.129-0.38.h5', 'weights.13-0.39.h5',

In [10]:
import tensorlayer as tl
from tensorflow.python.keras import losses
scores =[]
for mod in onlyfiles:
    if "weights" in mod:
        model = tf.keras.models.load_model(f"/{dirname}/{mod}",custom_objects={'<lambda>': lambda x : tl.act.lrelu(x, 0.1),"constrained_MSE":constrained_MSE})
        prediction_validation = np.array(model.predict(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values),dtype=float)
        scores.append(scoring.rejection90(validation.label, prediction_validation))
        print ("model: ",mod, " score: ",scores[-1])

model:  weights.559-4.03.h5  score:  0.6122303863474757
model:  weights.132-3.93.h5  score:  0.6286797819388481
model:  weights.988-4.07.h5  score:  0.6014221379473809
model:  weights.819-4.05.h5  score:  0.6133206921071344
model:  weights.294-3.98.h5  score:  0.5844986963735482
model:  weights.284-3.98.h5  score:  0.5989570988385874
model:  weights.747-4.03.h5  score:  0.6042664138421426
model:  weights.108-3.94.h5  score:  0.606257406968476
model:  weights.794-4.06.h5  score:  0.6164019909931263
model:  weights.118-3.94.h5  score:  0.6131784783123964
model:  weights.923-4.08.h5  score:  0.6210950462194833
model:  weights.287-3.96.h5  score:  0.6020383977245792
model:  weights.509-4.01.h5  score:  0.5961602275420715
model:  weights.420-4.02.h5  score:  0.5978193884806826
model:  weights.999-4.06.h5  score:  0.6101445840246504
model:  weights.286-3.97.h5  score:  0.6133680967053804
model:  weights.119-3.95.h5  score:  0.5822706802559848
model:  weights.859-4.07.h5  score:  0.6033657264

model:  weights.888-4.08.h5  score:  0.6036027494666982
model:  weights.226-3.96.h5  score:  0.6214742830054515
model:  weights.952-4.03.h5  score:  0.6123726001422138
model:  weights.942-4.03.h5  score:  0.6149798530457454
model:  weights.45-3.94.h5  score:  0.5844038871770562
model:  weights.501-4.01.h5  score:  0.6165442047878644
model:  weights.469-4.01.h5  score:  0.6263569566247926
model:  weights.55-3.94.h5  score:  0.6128940507229201
model:  weights.626-4.02.h5  score:  0.6203839772457929
model:  weights.677-4.01.h5  score:  0.5980090068736668
model:  weights.101-3.95.h5  score:  0.6303863474757051
model:  weights.54-3.95.h5  score:  0.6074899265228727
model:  weights.742-4.11.h5  score:  0.6161649680018961
model:  weights.461-3.99.h5  score:  0.6346053567196018
model:  weights.582-4.06.h5  score:  0.5902346527613178
model:  weights.100-3.94.h5  score:  0.6336572647546812
model:  weights.413-4.04.h5  score:  0.6118037449632614
model:  weights.475-4.02.h5  score:  0.604124200047

KeyboardInterrupt: 

In [ ]:
print (max(scores))

In [25]:
DATA_PATH = "/Users/mykola/MLHEP/MLHEP-2020-muon-id"
#DATA_PATH = dirname


test = utils.load_full_test_csv(DATA_PATH, "test-features.csv.gz")

test_closest_hits = test.apply(utils.find_closest_hit_per_station,axis=1)
Xarray_test = np.stack(test_closest_hits)
Closest_hits_test = pd.DataFrame(data=Xarray_test, index = range(len(test)), columns = FOI_Names)
Closest_hits_test.to_csv("big_test_transformed_foi.csv")

#print (Closest_hits)
nn_test_input = pd.concat([test, Closest_hits_test], axis=1, sort=False)
print(nn_test_input.shape)

/usr/local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(3630471, 73)


In [16]:
test = pd.read_csv(os.path.join(DATA_PATH, "test-features.csv.gz"),index_col="id", usecols=["id"]+utils.SIMPLE_FEATURE_COLUMNS)
foi_test = pd.read_csv(os.path.join(DATA_PATH, "big_test_transformed_foi.csv"))
nn_test_input = pd.concat([test, foi_test], axis=1, sort=False)

/usr/local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
print (nn_test_input)
nn_test_input.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names] = sc.transform(nn_test_input.loc[:, utils.SIMPLE_FEATURE_COLUMNS+FOI_Names])


         ncl[0]  ncl[1]  ncl[2]  ncl[3]  avg_cs[0]  avg_cs[1]  avg_cs[2]  \
0            55      12      10      15   2.127273   1.166667   1.500000   
1            43      18      12      15   2.697675   1.111111   1.333333   
2            90      34      19      29   1.800000   2.058824   1.210526   
3           141      43      15      16   4.609929   2.744186   1.466667   
4            22      18       8       9   1.636364   2.444444   1.000000   
...         ...     ...     ...     ...        ...        ...        ...   
3630466      19       6      23       8   1.736842   1.333333   1.391304   
3630467      42      10       6      18   2.309524   2.100000   2.166667   
3630468      36       6       8      25   1.638889   1.666667   1.250000   
3630469      90      31      10      15   2.888889   2.387097   1.300000   
3630470      61      18       8      14   1.737705   1.388889   1.000000   

         avg_cs[3]  ndof  MatchedHit_TYPE[0]  ...  closest_z_2  closest_z_3  \
0       

In [34]:
model_name = "weights.999-0.37.h5"
#  
model = tf.keras.models.load_model(f"{dirname}/{model_name}",custom_objects={'<lambda>': lambda x : tl.act.lrelu(x, 0.1),})
print ("model loaded")
predictions = np.array(model.predict_proba(nn_test_input[utils.SIMPLE_FEATURE_COLUMNS+FOI_Names].values),dtype=float)
print (predictions)


model loaded
[[0.89707601]
 [0.89613426]
 [0.96014762]
 ...
 [0.99611664]
 [0.88737953]
 [0.03571945]]


In [35]:
predictions=np.squeeze(np.asarray(predictions))
print (predictions)

[0.89707601 0.89613426 0.96014762 ... 0.99611664 0.88737953 0.03571945]


In [36]:
compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "submission.zip", index_label=utils.ID_COLUMN, compression=compression_opts)